In [15]:
# Importações

import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
from sklearn import preprocessing
import numpy as np

Adicionando as configurações de diretório e outros artefatos para o modelo

In [5]:
# diretorio da localização das imagens de treino
DIR = r"C:\Users\perei\Desktop\universidade\inteligencia artificial\utfpr_ia"
CATEGORIAS = ['mask', 'no_mask']

# iniciando os arrays de Data e labels vazios
data = []
labels = []

Adicionando as imagens de treino

In [6]:
for categoria in CATEGORIAS:
    path = os.path.join(DIR, categoria)
    for imagem in os.listdir(path):
        image_path = os.path.join(path, imagem)
        image = keras.preprocessing.image.load_img(image_path, target_size=(224,224))
        image = keras.preprocessing.image.img_to_array(image)
        image = keras.applications.mobilenet_v2.preprocess_input(image)

        data.append(image)
        labels.append(categoria)

C:\Users\perei\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Transformando valores categóricos em vetores binários esparsos

In [ ]:
bin_labels = preprocessing.LabelBinarizer()
labels = bin_labels.fit_transform(labels)
labels = keras.utils.to_categorical(labels)

Convertendo os arrays de data e labels para o formato float32 com o numpy

In [16]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

array([[[[-0.45098037, -0.49019605, -0.44313723],
         [-0.45098037, -0.49019605, -0.44313723],
         [-0.45098037, -0.49019605, -0.44313723],
         ...,
         [-0.41176468, -0.62352943, -0.75686276],
         [-0.41176468, -0.62352943, -0.75686276],
         [-0.41176468, -0.62352943, -0.75686276]],

        [[-0.45098037, -0.49019605, -0.44313723],
         [-0.45098037, -0.49019605, -0.44313723],
         [-0.45098037, -0.49019605, -0.44313723],
         ...,
         [-0.41176468, -0.62352943, -0.75686276],
         [-0.41176468, -0.62352943, -0.75686276],
         [-0.41176468, -0.62352943, -0.75686276]],

        [[-0.3960784 , -0.45098037, -0.3960784 ],
         [-0.3960784 , -0.45098037, -0.3960784 ],
         [-0.3960784 , -0.45098037, -0.3960784 ],
         ...,
         [-0.45098037, -0.62352943, -0.8039216 ],
         [-0.45098037, -0.62352943, -0.8039216 ],
         [-0.45098037, -0.62352943, -0.8039216 ]],

        ...,

        [[-0.46666664, -0.67058825, -0